In [42]:
from multiprocessing import Pool

import EEGAnalysis as ea
import h5py
from scipy.io import savemat, loadmat
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import re, os
from tqdm import tqdm

In [5]:
patient_id = 'zhangchen'

ea_manager = ea.DataManager('/media/STORAGE/EEG/Data')  ## create data manager
patient = ea_manager.create_patient(patient_id)  ## create new patient
# patient = ea_manager.get_patient(patient_id)  ## load previous patient

##### Constants #####
_freq = 2000
ROI = (-3, 3)
# frange = np.logspace(np.log10(10), np.log10(150), 40)
frange = np.logspace(np.log10(1), np.log10(150), 40)
zbase = (-2,-1)
tspec = np.linspace(ROI[0], ROI[1], int(_freq)*(ROI[1]-ROI[0]))


nyq = _freq / 2.0
width = 5.0 / nyq
ripple_db = 60.0

# Compute the order and Kaiser parameter for the FIR filter.
N, beta = signal.kaiserord(ripple_db, width)

taps = signal.firwin(N, [50/nyq, 80/nyq],window=('kaiser', beta),pass_zero=False)

# _filted = signal.filtfilt(taps, 1, _data)
# _hilbert_pwr = np.abs(signal.hilbert(_filted))

In [43]:
_total_ch = len(patient._sgch_config['chidx'])
export_name = './export/zhangchen_pwr_5-1'

interval = '5'  # '5' or '10'

_raw_trace = {}

# if not os.path.isdir(export_dir):
#     os.mkdir(export_dir)
#     os.chmod(export_dir, 0o775)

pbar = tqdm(total=len(_data_list.keys())*_total_ch*20)
for idate in _data_list.keys():
    date_name = 'd%s'%(idate.replace('-','p'))
    
    if interval == '5':
        _marker = patient.get_marker(file=idate)[20:]
    elif interval == '10':
        _marker = patient.get_marker(file=idate)[:20]
    else:
        raise ValueError("unkown paradigm code: %s"%paradigm)
    
    _result = np.zeros((20, _total_ch, np.size(tspec)))
    
    for chidx in range(_total_ch):
        _entry = patient.load_isplit(chidx, idate)[idate]['value']
        
        _filtered = signal.hilbert(signal.filtfilt(taps, 1, _data_list['190316-1']['value']))
        _epoch = ea.create_1d_epoch_bymarker(_filtered, _m, ROI, int(_freq))
        
        for i in range(20):
            _pwr = ea.decomposition.dwt_power(_epoch[i].reshape((1,1,-1)), _freq, baseline=(zbase[0]-ROI[0], zbase[1]-ROI[0]))
            _result[i, chidx, :] = _pwr[0]
            pbar.update(1)
        
    _raw_trace[date_name] = _result
        
pbar.close()

# ================ export ===============
savemat(export_name+'.mat', _raw_trace, do_compression=True)

with h5py.File(export_name+'.h5', 'w') as _f:
    for key, val in _raw_trace.items():
        _f.create_dataset(key, data=val, compression=9)

100%|██████████| 38220/38220 [40:21<00:00, 15.78it/s] 
